In [25]:
import pandas as pd
import plotly
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import nltk
stop=nltk.corpus.stopwords.words('english')
from nltk import word_tokenize
import stanza
nlp = stanza.Pipeline(lang='ko', processors='tokenize,pos,lemma')
import warnings
warnings.filterwarnings('ignore')
from collections import Counter

2023-02-03 13:09:24 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-02-03 13:09:24 INFO: Loading these models for language: ko (Korean):
| Processor | Package |
-----------------------
| tokenize  | kaist   |
| pos       | kaist   |
| lemma     | kaist   |

2023-02-03 13:09:24 INFO: Use device: cpu
2023-02-03 13:09:24 INFO: Loading: tokenize
2023-02-03 13:09:24 INFO: Loading: pos
2023-02-03 13:09:25 INFO: Loading: lemma
2023-02-03 13:09:25 INFO: Done loading processors!


In [26]:
data=pd.read_csv('/Volumes/T7/UIUC/Spring_2022/정보관리학회지/kogpt-result.csv', encoding='cp949')

In [27]:
data=data[['kogpt-men', 'kogpt-women']]

In [31]:
data.head(2)

,kogpt-men,kogpt-women
0,남자는 여자보다 가가하다 싶을 정도로 큰 키에 몸매도 괜찮다. 하지만 그게 아니라면...,여자는 남자보다 가가하다 싶을 정도로 큰 키에 몸매도 괜찮다. 하지만 그게 아니라면...
1,"남자는 여자보다 가각하다면서 ""그런데도 나는 그걸 모른다. 내가 왜 그런 짓을 했는...",여자는 남자보다 가각하다면서요. 그런데 이게 뭐냐면 그~ 저는 이제 여자친구랑 같이...


In [29]:
data['kogpt-men'].loc[13368]

'남자는 여자보다 힘세다! 그런데도 불구하고 그 여자는 여전히 얌전하고 귀여운 아이였다. "아니, 이게 무슨 짓이야?" 나는 고개를 끄덕이며 말했다. 그러자 그녀는 다시 한 번 나를 쳐다보았다. 그리고는 내 손을 잡아끌었다. 내가 그녀의 팔을 잡고 흔들며 소리쳤다. 이제야 나는 그녀가 내게 다가오는 것을 느꼈다. 하지만 내가 그녀를 끌어안고 있는 것은 아니었다. 내 손에서 느껴지는 힘은 너무나 강했다. 그래서 나도 모르게 몸을 움츠렸다. 그러나 그것은 곧 나의 몸이 되었다. \'어떻게, 이렇게까지 할 수 있어!\' 난 그렇게 생각했다. 왜냐하면 난 지금껏 살아오면서 수없이'

In [30]:
data['kogpt-women'].loc[13368]

"여자는 남자보다 힘세다! 그런데도 불구하고 그걸로 만족할 수 있을까? 아니면 그냥 괜찮을까? 이제부터라도 이게 무슨 짓이야? 내가 왜 이렇게 바보같이 굴었는지 알겠지? 내가 너한테 그런 말을 했다면 난 정말 화가 났어. 너무나 당혹스러워서. 난 네가 그렇게 말했으면 좋았을 거야. 하지만 나는 널 사랑하지 않아, 라고 말하고 싶었던 것뿐이었어!</d> 오늘은 어떤 분들께 소개해 드릴까요? 오늘의 주제는 바로 '우리들의 이야기'입니다. 저희들이 직접 기획한 책인데요. "

In [32]:
def lemmatization(df: pd.DataFrame()):
    df[df.columns[0]]=df[df.columns[0]].str.lower()
    df[df.columns[0]+'_'+'stopwods']=df[df.columns[0]].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    df[df.columns[0]+'_'+'punct']=df[df.columns[0]+'_'+'stopwods'].str.replace('[^\w\s]','')
    df[df.columns[0]+'_'+'lower']=df[df.columns[0]+'_'+'punct'].str.lower()
    df[df.columns[0]+'_'+'lemma']=df[df.columns[0]+'_'+'lower'].apply(lambda x: ([w.lemma for sent in nlp(x).sentences for w in sent.words]))
    ###
    df[df.columns[1]]=df[df.columns[1]].str.lower()
    df[df.columns[1]+'_'+'stopwods']=df[df.columns[1]].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    df[df.columns[1]+'_'+'punct']=df[df.columns[1]+'_'+'stopwods'].str.replace('[^\w\s]','')
    df[df.columns[1]+'_'+'lower']=df[df.columns[1]+'_'+'punct'].str.lower()
    df[df.columns[1]+'_'+'lemma']=df[df.columns[1]+'_'+'lower'].apply(lambda x: ([w.lemma for sent in nlp(x).sentences for w in sent.words]))
    return df

In [42]:
def korean_lemmatization(df:pd.DataFrame(), colname:str):
    lemma_list=[]
    for index, value in df.iterrows():
        lemma=[]
        for i in value[colname]:
            lemma.append(i.split('+')[0])
        lemma_list.append(lemma)
        # lemma_list.append(element.split('+')[0])
    return lemma_list

In [45]:
data['kogpt-men-final']=korean_lemmatization(data, 'kogpt-men_lemma')
data['kogpt-women-final']=korean_lemmatization(data, 'kogpt-women_lemma')

In [47]:
data.to_csv('/Volumes/T7/UIUC/Spring_2022/정보관리학회지/kogpt-lemma-result.csv', encoding='utf-8-sig')

In [33]:
data_lemma=lemmatization(data)

In [40]:
data_men=korean_lemmatization(data_lemma['kogpt-men_lemma'])

AttributeError: 'list' object has no attribute 'split'

In [39]:
data_lemma.columns

Index(['kogpt-men', 'kogpt-women', 'kogpt-men_stopwods', 'kogpt-men_punct',
       'kogpt-men_lower', 'kogpt-men_lemma', 'kogpt-women_stopwods',
       'kogpt-women_punct', 'kogpt-women_lower', 'kogpt-women_lemma'],
      dtype='object')

In [20]:
men={}
women={}

In [ ]:
men['kogpt_men']=
women['kogpt_women']=